In [40]:
import numpy as np

In [41]:
def data_generator(beta, typ):
    p = beta.shape[0]
    X = np.ones(p)
    X[1:] = np.random.normal(0, 1, size = p - 1)
    u = X.dot(beta)
    if typ == "Linear":
        return X, u + np.random.normal(0, 0.1, 1)
    elif typ == "Logistic":
        return X, np.random.binomial(1, 1/(1+np.exp(-u)))

In [42]:
def loss(beta, data, typ):
    X, Y = data
    u = X.dot(beta)
    if typ == "Linear":
        return (Y - u)**2/2
    elif typ == "Logistic":
        mu = 1/(1+np.exp(-u))
        return -Y * np.log(mu) - (1 - Y) * np.log(1 - mu)

In [43]:
def grad(beta, data, typ):
    X, Y = data
    u = X.dot(beta)
    if typ == "Linear":
        return (u - Y) * X
    elif typ == "Logistic":
        mu = 1/(1+np.exp(-u))
        return (mu - Y) * X

In [58]:
def second(beta, data, typ):
    X, Y = data
    u = X.dot(beta)
    if typ == "Linear":
        g = (u - Y) * X
        S = np.outer(g, g)
        H = np.outer(X, X)
        return((S, H))
    elif typ == "Logistic":
        mu = 1/(1+np.exp(-u))
        g = (mu - Y) * X
        S = np.outer(g, g)
        H = np.outer(X * mu * (1 - mu), X)
        return((S, H))

In [66]:
def sgd(T, beta, beta_true, alpha, gamma, typ):
    beta_bar = np.zeros_like(beta)
    Sig = 0
    Hes = 0
    Y = 0
    Ysq = 0
    for t in range(T):
        data = data_generator(beta_true, typ)
        Y += data[1]
        Ysq += data[1]**2
        l = loss(beta, data, typ)
        g = grad(beta, data, typ)
        S, H = second(beta, data, typ)
        Sig += S
        Hes += H
        beta -= (t + 1) ** (-gamma) * alpha * g
        beta_bar *= t/(t + 1)
        beta_bar += beta/(t + 1)
    Sig /= T
    Hes /= T
    Y /= T
    Ysq /= T
    Hes_inv = np.linalg.inv(Hes)
    pvar = np.dot(np.dot(Hes_inv, Sig), Hes_inv)
    pse = np.sqrt(np.diag(pvar)/T)
    vse = np.sqrt((Ysq - Y**2)/T)
    return beta_bar, pse, Y, vse

In [72]:
def mc(M, T, beta, beta_true, alpha, gamma, typ):
    p = beta.shape[0]
    beta_log = np.empty((M, p))
    pse_log = np.empty((M, p))
    Y_log = np.empty(M)
    vse_log = np.empty(M)
    for m in range(M):
        beta_bar, pse, Y, vse = sgd(T, beta, beta_true, alpha, gamma, typ)
        beta_log[m, :] = beta_bar
        pse_log[m, :] = pse
        Y_log[m] = Y
        vse_log[m] = vse
    return np.mean(pse_log, axis = 0)/np.std(beta_log, axis = 0), np.mean(vse_log)/np.std(Y_log)

In [76]:
for T in [100, 500, 1000]:
    print(mc(200, T, np.zeros(3), np.array([0.3, -0.1, 0.7]), 0.5, 0.501, "Linear"))

(array([0.97827632, 0.87414142, 0.8846496 ]), 0.9429592545261877)
(array([1.16978711, 1.06868078, 1.07280728]), 1.0115496662633467)
(array([1.06868286, 0.9914795 , 0.99409808]), 0.9948434306226355)


In [78]:
for T in [500, 1000, 5000]:
    print(mc(200, T, np.zeros(3), np.array([0.3, -0.1, 0.7]), 0.5, 0.501, "Logistic"))

(array([0.99927028, 1.0236824 , 0.90247199]), 1.0708423137903147)
(array([0.92402511, 0.98303095, 0.92732851]), 1.1115664632072142)
(array([0.96507403, 1.03374832, 1.01392052]), 0.9343051232878505)


In [79]:
for T in [10000]:
    print(mc(200, T, np.zeros(3), np.array([0.3, -0.1, 0.7]), 0.5, 0.501, "Logistic"))

(array([0.97603486, 0.91619487, 0.97325383]), 1.0364293186300684)
